# Set up Elasticsearch.

## Index the movies.

In [12]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.1 MB 2.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached backcall-0.1.0.tar.gz (9.7 kB)
     |████████████████████████████████| 151 kB 19.0 MB/s eta 0:00:01
     |████████████████████████████████| 157 kB 35.6 MB/s eta 0:00:01
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 80 kB 22.4 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 40.3 MB/s eta 0:00:01
  Using cached joblib-0

In [11]:
from ltr import download
download()

from ltr.client import ElasticClient
client = ElasticClient()

from ltr.index import rebuild_tmdb
rebuild_tmdb(client)

ModuleNotFoundError: No module named 'ltr'

In [ ]:
config = {
    "featureset": {
        "features": [
            {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                            "field": "release_year",
                            "missing": 2000
                        },
                        "query": { "match_all": {} }
                    }
                }
            }
        ]
    }
}

from ltr import setup
setup(client, config=config, index='tmdb', featureset='release')

from ltr import years_as_ratings
years_as_ratings.synthesize(client,
                            featureSet='release',
                            classicTrainingSetOut='data/classic-training.txt',
                            latestTrainingSetOut='data/latest-training.txt')

from ltr import train
train(client, trainingInFile='data/latest-training.txt',
      index='tmdb', featureSet='release', modelName='latest')
train(client, trainingInFile='data/classic-training.txt',
      index='tmdb', featureSet='release', modelName='classic')

from ltr.release_date_plot import plot
plot(client)

## Search for a movie:

In [10]:
%%bash
curl -s -X GET "http://localhost:9200/_search" -H "Content-Type: application/json" -d'
{
    "query": {
        "multi_match" : {
            "query" : "rambo",
            "fields" : ["overview", "tagline"]            
        }
    }
}'

CalledProcessError: Command 'b'curl -s -X GET "http://localhost:9200/_search" -H "Content-Type: application/json" -d\'\n{\n    "query": {\n        "multi_match" : {\n            "query" : "rambo",\n            "fields" : ["overview", "tagline"]            \n        }\n    }\n}\'\n'' returned non-zero exit status 7.